### This will be a capstone project

In [42]:
import pandas as pd
import numpy as np
import requests
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge geocoder --yes

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder

!conda install --yes  beautifulsoup4


from bs4 import BeautifulSoup
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library



Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge

The following packages will be UPDATED:

    certifi: 2019.9.11-py36_0  --> 2019.9.11-py36_0  conda-forge

The following packages will be DOWNGRADED:

    openssl: 1.1.1d-h7b6447c_3 --> 1.1.1c-h516909a_0 conda-forge


certifi-2019.9.11    | 147 KB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done
Solving environment: done


==> WARNING: A newer version of cond

In [43]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

source = requests.get('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago').text

soup = BeautifulSoup(source, 'html5lib')

In [44]:
commArea = []
neighborhoodList = []

In [45]:
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        neighborhoodList.append(cells[0].text.rstrip('\n'))
        commArea.append(cells[1].text.rstrip('\n'))

In [46]:
chi_df = pd.DataFrame({"Neighborhood": neighborhoodList, "Community": commArea})

In [47]:
chi_df.head(5)

,Neighborhood,Community
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [48]:
chi_df.shape

(246, 2)

In [49]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Chicago, Illinois'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [50]:
coords = [ get_latlng(neighborhood) for neighborhood in chi_df["Neighborhood"].tolist() ]

In [51]:
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])


In [52]:
chi_df['Latitude'] = df_coords['Latitude']
chi_df['Longitude'] = df_coords['Longitude']

In [53]:
chi_df.head(5)

,Neighborhood,Community,Latitude,Longitude
0,Albany Park,Albany Park,41.96829,-87.72338
1,Altgeld Gardens,Riverdale,41.65441,-87.60225
2,Andersonville,Edgewater,41.98046,-87.66834
3,Archer Heights,Archer Heights,41.81154,-87.72556
4,Armour Square,Armour Square,41.83458,-87.63189


In [56]:
address = 'Chicago, Illinois'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago, Illinois {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Chicago, Illinois 41.8755616, -87.6244212.


In [57]:
# create map of New York using latitude and longitude values
map_chicago = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(chi_df['Latitude'], chi_df['Longitude'],chi_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_chicago)  
    
map_chicago

NameError: name 'neighborhoods' is not defined